In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import display

movies_data_path = "/content/drive/MyDrive/movies.csv"
df_movies = pd.read_csv(movies_data_path, sep=",")


ratings_path = "/content/drive/MyDrive/ratings.csv"
df_raitings = pd.read_csv(ratings_path, sep=",")

df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df_raitings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load data
df_movies = pd.read_csv("/content/drive/MyDrive/movies.csv")
df_raitings = pd.read_csv("/content/drive/MyDrive/ratings.csv")

# Filter to first 200 users and their ratings
users_ids = sorted(df_raitings['userId'].unique())[:200]
ratings = df_raitings[df_raitings['userId'].isin(users_ids)]

# Create user-item matrix (movieId as rows, userId as columns)
user_item_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating').fillna(0)

# Calculate item-item cosine similarity
item_similarity = cosine_similarity(user_item_matrix)
item_similarity_df = pd.DataFrame(item_similarity,
                                  index=user_item_matrix.index,
                                  columns=user_item_matrix.index)

# Movie metadata after filtering for movies in the similarity matrix
movie_ids = user_item_matrix.index.tolist()
movies = df_movies[df_movies['movieId'].isin(movie_ids)]

# Function to get top similar movies using the similarity matrix
def get_top_similar_movies(movie_id, similarity_df, movies_df, top_n=10):
    if movie_id not in similarity_df.index:
        return pd.DataFrame(columns=['movieId', 'title'])
    similar_movies = similarity_df[movie_id].sort_values(ascending=False).iloc[1:top_n+1]
    return movies_df[movies_df['movieId'].isin(similar_movies.index)][['movieId', 'title']]

# Print top similar movies
print("Top 10 similar movies to Toy Story (1995):")
toy_story_similar = get_top_similar_movies(1, item_similarity_df, movies)
print(toy_story_similar)

print("\nTop 10 similar movies to Waiting to Exhale (1995):")
waiting_similar = get_top_similar_movies(4, item_similarity_df, movies)
print(waiting_similar)




Top 10 similar movies to Toy Story (1995):
      movieId                                      title
224       260  Star Wars: Episode IV - A New Hope (1977)
257       296                        Pulp Fiction (1994)
325       367                           Mask, The (1994)
418       480                       Jurassic Park (1993)
506       588                             Aladdin (1992)
546       648                 Mission: Impossible (1996)
615       780       Independence Day (a.k.a. ID4) (1996)
964      1265                       Groundhog Day (1993)
2355     3114                         Toy Story 2 (1999)
3194     4306                               Shrek (2001)

Top 10 similar movies to Waiting to Exhale (1995):
     movieId                                   title
100      113                 Before and After (1996)
232      270                      Love Affair (1994)
250      289                         Only You (1994)
262      302  Queen Margot (Reine Margot, La) (1994)
312      354 